In [109]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import mysql.connector


def setup_driver(driver_path):
    """Set up and return a Selenium WebDriver instance."""
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    driver.implicitly_wait(5)
    return driver

In [110]:
driver_path = r'C:\Users\AAA\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
driver = setup_driver(driver_path)


In [111]:
def setup_db_connection():
    """Set up and return a MySQL database connection."""
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Byy200402191017",
        database="report",
        charset="utf8mb4"
    )
    return conn


In [112]:
conn = setup_db_connection()


In [113]:
def collect_links(driver, url, number):
    """Collect a specified number of unique links from the given URL."""
    links = set()
    old_count = 0

    driver.get(url)

    while len(links) < number:
        news_items = driver.find_elements(
            By.CSS_SELECTOR,
            "#nimbus-app > section > section > section > article > section.mainContent.yf-tnbau3 > section > "
            "div > div > div > div > ul > li > section > div > a"
        )

        for item in news_items:
            href = item.get_attribute("href")
            if href and href not in links:
                links.add(href)

        if len(links) >= number:
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        if len(links) == old_count:
            print("No more new links found. Stopping...")
            break
        old_count = len(links)

    return list(links)[:number]

In [114]:
url = f'https://finance.yahoo.com/quote/MS/news/'
number = 5
links = collect_links(driver, url, number)
print("Collected Links:", links)
len(links[0])


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF729B6FB05+28789]
	(No symbol) [0x00007FF729AD86E0]
	(No symbol) [0x00007FF72997592A]
	(No symbol) [0x00007FF72994F505]
	(No symbol) [0x00007FF7299F6477]
	(No symbol) [0x00007FF729A0EF42]
	(No symbol) [0x00007FF7299EF1E3]
	(No symbol) [0x00007FF7299BA938]
	(No symbol) [0x00007FF7299BBAA1]
	GetHandleVerifier [0x00007FF729EA933D+3410093]
	GetHandleVerifier [0x00007FF729EBE7DD+3497293]
	GetHandleVerifier [0x00007FF729EB2A73+3448803]
	GetHandleVerifier [0x00007FF729C37BBB+848171]
	(No symbol) [0x00007FF729AE3C3F]
	(No symbol) [0x00007FF729ADF6E4]
	(No symbol) [0x00007FF729ADF87D]
	(No symbol) [0x00007FF729ACED49]
	BaseThreadInitThunk [0x00007FF94779259D+29]
	RtlUserThreadStart [0x00007FF948D2AF38+40]


In [62]:
len(links[1])

74

In [116]:
import requests
from bs4 import BeautifulSoup

def fetch_article_content(url):
    try:
        
        response = requests.get(url, timeout=10)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, "html.parser")
        
       
        paragraphs = soup.find_all('p', class_='yf-1pe5jgt')  
        content = []

        for para in paragraphs:
           
            para_text = []
            for elem in para.contents:
                if elem.name == 'a':  
                    link_text = f"{elem.get_text(strip=True)} ({elem['href']})"
                    para_text.append(link_text)
                else:  
                    para_text.append(elem.strip() if isinstance(elem, str) else elem.get_text(strip=True))
            content.append(" ".join(para_text))
        
        return "\n".join(content)
    
    except Exception as e:
        return f"Error fetching content: {e}"
def fetch_article_links(links):
    result = []
    for link in links:
        content = fetch_article_content(link)
        if content:
            result.append([content])
    return result 

def fetch_non_empty_links(links):
    result_links = []
    for link in links:
        content = fetch_article_content(link)
        if content:
            result_links.append([link])
    return result_links
        




In [132]:
list_content = fetch_article_links(links)
list_links = fetch_non_empty_links(links)
print(len(list_content))


4


In [141]:
list_links[1]

['https://finance.yahoo.com/news/visa-morgan-stanley-top-pick-144647813.html']

In [140]:
list_content[1]

["HTML_TAG_START Investing.com --Morgan Stanley named Visa Inc (NYSE:V) its top stock pick in the payments sector for 2025, given its strong growth prospects from international travel, value-added services, and new payment flows. HTML_TAG_END\nHTML_TAG_START The brokerage remains overweight on Visa and Mastercard (NYSE:MA), favouring their defensible business models, strong operating leverage, and limited competition. HTML_TAG_END\nHTML_TAG_START “We are adding V as our Top Pick into 2025 given attractive valuation, benefits from travel + value added services, easing regulatory scrutiny, and favorable tactical trading dynamics,” analyst said. HTML_TAG_END\nHTML_TAG_START Morgan Stanley (NYSE:MS) expects both companies to deliver double-digit earnings growth with potential for modest valuation expansion. HTML_TAG_END\nHTML_TAG_START Visa's valuation appears particularly attractive on a growth-adjusted basis compared to consumer peers, the note said. Morgan Stanley also highlighted reduc

In [117]:
import re

def clean_list_of_list_content(raw_contents):
    """
    Cleans a list of list content by removing placeholders, unnecessary whitespace, 
    and formatting for readability.
    
    Args:
        raw_contents (list of lists): Each sublist contains a string of raw content.
    
    Returns:
        list of lists: Each sublist contains a single cleaned and formatted string.
    """
    cleaned_contents = []
    
    for sublist in raw_contents:
        # Check if sublist has content
        if not sublist or not isinstance(sublist[0], str):
            cleaned_contents.append(["No valid content provided."])
            continue

        raw_text = sublist[0]

       
        cleaned_text = re.sub(r'HTML_TAG_START|HTML_TAG_END', '', raw_text)

        
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

        
        paragraphs = cleaned_text.split('. ')
        formatted_content = "\n\n".join(paragraphs)
        
        
        cleaned_contents.append([formatted_content])
    
    return cleaned_contents




In [86]:
clean_list_of_list_content(list_content)

[['NEW YORK, December 18, 2024 --( BUSINESS WIRE (https://www.businesswire.com) )--Morgan Stanley Investment Management, through investment funds managed by Morgan Stanley Real Estate Investing (MSREI), announced today it has partnered with Brightview Senior Living (Brightview) to acquire a portfolio of eight premium senior housing communities from Harrison Street, one of the leading investment management firms exclusively focused on alternative real assets with approximately $56 billion in assets under management\n\nThe portfolio, which Brightview originally developed, is comprised of 1,186 units across the Baltimore, Philadelphia, Providence and Boston metropolitan areas\n\nBrightview will continue to operate the communities\n\n"We are pleased to acquire this high-quality portfolio of senior housing communities that have demonstrated a track record of strong operational performance," said Will Milam, Head of U.S\n\nInvestments at Morgan Stanley Real Estate Investing\n\n"Aging demogra

In [144]:
'''
def insert_content_to_db(conn, record_id, list_content, non_empty_links):
    """Insert content into the database."""
    cursor = conn.cursor()
    try:
        for i, content_list in enumerate(list_content):
            if not content_list:  
                print(f"No content for News {i}, skipping...")
                continue

            content = " ".join(content_list)
            content = content.encode("utf-8", "ignore").decode("utf-8")

            sql_insert = """
                INSERT INTO NEWS (Record_id, New_link, content)
                VALUES (%s, %s, %s);
            """
            cursor.execute(sql_insert, (record_id, content, non_empty_links[i]))
            conn.commit()
            print(f"Data inserted successfully for News {i+1}")
    except Exception as e:
        conn.rollback()
        print("Error while inserting data:", e)
    finally:
        cursor.close()

In [146]:
record_id = 1
insert_content_to_db(conn, record_id, list_content,list_links)
'''

Error while inserting data: Python type list cannot be converted


In [149]:
def insert_content_to_db(conn, record_id, list_content, non_empty_links):
    """Insert content and links into the database."""
    cursor = conn.cursor()
    try:
        for i, (content_list, link) in enumerate(zip(list_content, non_empty_links)):
            
            if not content_list:  
                print(f"No content for News {i+1}, skipping...")
                continue

            content = " ".join(
                item if isinstance(item, str) else str(item) for item in content_list
            )
            content = content.encode("utf-8", "ignore").decode("utf-8")

            
            if isinstance(link, list):
                link = " ".join(link)  
            link = link.encode("utf-8", "ignore").decode("utf-8")  

            
            sql_insert = """
                INSERT INTO NEWS (Record_id, News_link, content)
                VALUES (%s, %s, %s);
            """
           
            cursor.execute(sql_insert, (record_id, link, content))
            conn.commit()
            print(f"Data inserted successfully for News {i+1}")
    except Exception as e:
        conn.rollback()
        print("Error while inserting data:", e)
    finally:
        cursor.close()


In [150]:
record_id = 1
insert_content_to_db(conn, record_id, list_content,list_links)
    

Data inserted successfully for News 1
Data inserted successfully for News 2
Data inserted successfully for News 3
Data inserted successfully for News 4


In [121]:
def get_content(company, number, record_id):
    """Main function to get content from Yahoo Finance and store it in the database."""
    url = f'https://finance.yahoo.com/quote/{company}/news/'
    driver_path = r'C:\Users\AAA\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

    driver = setup_driver(driver_path)
    conn = setup_db_connection()

    try:
      
        links = collect_links(driver, url, number)
        print("Collected Links:", links)
        
        non_empty_links = fetch_non_empty_links(links)
        print("Non Empty Links:", non_empty_links)

        
        list_content = fetch_article_links(links)
        print(f"=== Collected {len(list_content)} Contents ===")

      
        insert_content_to_db(conn, record_id, list_content, non_empty_links)
    finally:
        driver.quit()
        conn.close()




In [151]:
get_content("MS", 5, 1)

Collected Links: ['https://finance.yahoo.com/news/morgan-stanley-real-estate-investing-140000492.html', 'https://finance.yahoo.com/news/visa-morgan-stanley-top-pick-144647813.html', 'https://finance.yahoo.com/news/e-trade-plans-zero-cost-192601537.html', 'https://finance.yahoo.com/news/robinhoods-surge-towards-profitability-brings-it-back-to-the-top-of-the-financial-world-110048551.html', 'https://finance.yahoo.com/m/341bebc2-1c07-3cac-8a5f-0193edc2db72/xerox-to-buy-printer-maker.html']
=== Collected 4 Contents ===
Data inserted successfully for News 1
Data inserted successfully for News 2
Data inserted successfully for News 3
Data inserted successfully for News 4
